# CUDA Kernels

We will use CuPy to execute CudaC Kernel to `add vectors` from the Python runtime

In [2]:
#enable T4 gpu in Runtime > Change Runtime Type
!nvidia-smi

Fri May  2 12:53:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# CUDA Kernel

In [3]:
kernel_code = r'''
extern "C" __global__
void matmul(const float* A, const float* B, float* C, int M, int K, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        float sum = 0.0f;
        for (int i = 0; i < K; ++i) {
            sum += A[row * K + i] * B[i * N + col];
        }
        C[row * N + col] = sum;
    }
}
'''

# Main

In [4]:
import cupy as cp

module = cp.RawModule(code=kernel_code)
matmul_kernel = module.get_function('matmul')

In [5]:
#mat dims
M, K, N = 256, 512, 128

#note (n1, n2).(n2, n3)  --->  (n1, n3)
A = cp.random.rand(M, K, dtype=cp.float32)
B = cp.random.rand(K, N, dtype=cp.float32)
C = cp.empty((M, N), dtype=cp.float32)

In [6]:
threads_per_block = (16, 16)
blocks_per_grid_x = (N + threads_per_block[0] - 1) // threads_per_block[0]
blocks_per_grid_y = (M + threads_per_block[1] - 1) // threads_per_block[1]

In [7]:
matmul_kernel(
    (blocks_per_grid_x, blocks_per_grid_y),
    threads_per_block,
    (A, B, C, M, K, N)
)

In [8]:
C_ref = A @ B
assert cp.allclose(C, C_ref, atol=1e-3)

In [11]:
print(C)
print(C_ref)

[[129.7565   129.64478  123.23375  ... 116.80481  124.080055 125.91232 ]
 [127.11189  129.48016  128.48605  ... 119.160095 127.22114  128.10782 ]
 [127.34656  132.18071  126.87031  ... 118.25388  124.63174  129.82968 ]
 ...
 [125.09992  124.69797  119.99035  ... 116.124664 126.137184 122.30461 ]
 [120.753235 127.84794  120.43405  ... 117.633224 120.90432  123.22717 ]
 [132.0097   136.14296  132.2939   ... 125.32579  126.92527  135.43048 ]]
[[129.75645  129.64482  123.23378  ... 116.80486  124.08003  125.912346]
 [127.11194  129.48009  128.48602  ... 119.160126 127.2211   128.10776 ]
 [127.34652  132.18071  126.870316 ... 118.253876 124.631714 129.82968 ]
 ...
 [125.09988  124.69794  119.9903   ... 116.124664 126.13722  122.30462 ]
 [120.75323  127.847946 120.43403  ... 117.63323  120.90431  123.227104]
 [132.00969  136.14297  132.29388  ... 125.32575  126.92525  135.43054 ]]
